# Titanic survival

First, we import libraries

In [409]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

### Data initialization

In [410]:
df = pd.read_csv('titanic.csv', sep=',')
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


From here, we know which data we want to keep and convert to numerical values.

In [411]:
df = df.drop(['PassengerId'], axis = 1)
df = df.drop(['Ticket'], axis = 1)
df = df.drop(['Name'], axis = 1)
df = df.drop(['Fare'], axis = 1)

df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Cabin,Embarked
0,0,3,male,22.0,1,0,NaN,S
1,1,1,female,38.0,1,0,C85,C
2,1,3,female,26.0,0,0,NaN,S
3,1,1,female,35.0,1,0,C123,S
4,0,3,male,35.0,0,0,NaN,S


In [412]:
deck = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "U": 8}

df['Cabin'] = df['Cabin'].str[0]
df['Deck'] = df['Cabin'].map(deck)
df['Deck'] = df['Deck'].fillna("0")

df = df.drop(['Cabin'], axis = 1)

df.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked,Deck
0,0,3,male,22.0,1,0,S,0
1,1,1,female,38.0,1,0,C,3.0


In [413]:
gender = {"male": 0, "female": 1}
df['Sex'] = df['Sex'].map(gender)

df.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked,Deck
0,0,3,0,22.0,1,0,S,0
1,1,1,1,38.0,1,0,C,3.0


In [414]:
place = {"C": 0, "S": 1, "Q": 2}
df['Embarked'] = df['Embarked'].map(place)
df['Embarked'] = df['Embarked'].fillna(1)

df.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked,Deck
0,0,3,0,22.0,1,0,1.0,0
1,1,1,1,38.0,1,0,0.0,3.0


In [415]:
age_mean = df['Age'].mean()
age_std = df['Age'].std()
df['Age'] = df['Age'].fillna(np.random.randint(age_mean - age_std, age_mean + age_std))


In [416]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Embarked  891 non-null    float64
 7   Deck      891 non-null    object 
dtypes: float64(2), int64(5), object(1)
memory usage: 55.8+ KB


In [417]:
X = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Embarked', 'Deck']].values
y = df['Survived'].values
print(np.shape(X))
print(np.shape(y))

(X_train, X_test, y_train, y_test) = train_test_split(X, y)

(891, 7)
(891,)


In [418]:
sgd = linear_model.SGDClassifier(max_iter=5, tol=None)
sgd.fit(X_train, y_train)
Y_pred = sgd.predict(X_test)

sgd.score(X_train, y_train)

acc_sgd = round(sgd.score(X_train, y_train) * 100, 2)

In [419]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)

Y_prediction = random_forest.predict(X_test)

random_forest.score(X_train, y_train)
acc_random_forest = round(random_forest.score(X_train, y_train) * 100, 2)

In [420]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

Y_pred = logreg.predict(X_test)

acc_log = round(logreg.score(X_train, y_train) * 100, 2)

In [421]:
knn = KNeighborsClassifier(n_neighbors = 3) 
knn.fit(X_train, y_train)  
Y_pred = knn.predict(X_test)  
acc_knn = round(knn.score(X_train, y_train) * 100, 2)

In [422]:
decision_tree = DecisionTreeClassifier() 
decision_tree.fit(X_train, y_train)  
Y_pred = decision_tree.predict(X_test)  
acc_decision_tree = round(decision_tree.score(X_train, y_train) * 100, 2)

In [423]:
results = pd.DataFrame({
    'Model': ['KNN', 'Logistic Regression', 
              'Random Forest', 
              'Stochastic Gradient Decent', 
              'Decision Tree'],
    'Score': [acc_knn, acc_log, 
              acc_random_forest, 
              acc_sgd, acc_decision_tree]})
result_df = results.sort_values(by='Score', ascending=False)
result_df = result_df.set_index('Score')
result_df.head()

,Model
Score,
94.76,Random Forest
94.76,Decision Tree
87.57,KNN
81.14,Logistic Regression
39.22,Stochastic Gradient Decent


In [425]:
rf = RandomForestClassifier(n_estimators=100)
scores = cross_val_score(rf, X_train, y_train, cv=10, scoring = "accuracy")
print("Mean RF:", scores.mean())
print("Standard Deviation RF:", scores.std())

dt = DecisionTreeClassifier()
scores = cross_val_score(dt, X_train, y_train, cv=10, scoring = "accuracy")
print("Mean DT:", scores.mean())
print("Standard Deviation DT:", scores.std())

Mean RF: 0.8038218000904568
Standard Deviation RF: 0.02766027497549411
Mean DT: 0.8007462686567164
Standard Deviation DT: 0.045156936978970066


In [426]:
feature_scores = pd.Series(np.round(random_forest.feature_importances_,3), index=df.drop('Survived', axis = 1).columns).sort_values(ascending=False)
feature_scores

Age         0.323
Sex         0.313
Deck        0.102
Pclass      0.092
SibSp       0.065
Parch       0.053
Embarked    0.052
dtype: float64